I do not understand why in this example they classify on the file - not the audio stream.

In [2]:
from datasets import load_dataset
from transformers import pipeline

dataset = load_dataset("anton-l/superb_demo", "er", split="session1")

classifier = pipeline("audio-classification", model="superb/hubert-large-superb-er")
labels = classifier(dataset[0]["file"], top_k=5)

Generating session1 split: 0 examples [00:00, ? examples/s]

Dataset superb_demo downloaded and prepared to /home/jovyan/.cache/huggingface/datasets/anton-l___superb_demo/er/1.9.0/77d23894ff429329a7fe80f9007cabb0deec321316f8dda1a1e9d10ffa089d08. Subsequent calls will reuse this data.


In [8]:
print(dataset[0]['file'])

/home/jovyan/.cache/huggingface/datasets/downloads/extracted/78e6da90361e53b909a57c3b6dc0334cd09f6bc67691afb514a961f5df6e9bdf/Session1/sentences/wav/Ses01F_impro03/Ses01F_impro03_F013.wav


In [6]:
print(labels[0])

[{'score': 0.9982258677482605, 'label': 'hap'}, {'score': 0.001702160807326436, 'label': 'sad'}, {'score': 4.696955147664994e-05, 'label': 'neu'}, {'score': 2.500531445548404e-05, 'label': 'ang'}]


In [10]:
print(dataset[0]['audio'])

{'path': '/home/jovyan/.cache/huggingface/datasets/downloads/extracted/78e6da90361e53b909a57c3b6dc0334cd09f6bc67691afb514a961f5df6e9bdf/Session1/sentences/wav/Ses01F_impro03/Ses01F_impro03_F013.wav', 'array': array([ 0.01025391,  0.00134277, -0.0039978 , ..., -0.00027466,
       -0.00231934, -0.00259399], dtype=float32), 'sampling_rate': 16000}


Why in this example do they use librosa to load the file into speech - why not use audio?

In [1]:
import torch
import librosa
from datasets import load_dataset
from transformers import HubertForSequenceClassification, Wav2Vec2FeatureExtractor

def map_to_array(example):
    speech, _ = librosa.load(example["file"], sr=16000, mono=True)
    example["speech"] = speech
    return example

# load a demo dataset and read audio files
dataset = load_dataset("anton-l/superb_demo", "er", split="session1")
dataset = dataset.map(map_to_array)


Reusing dataset superb_demo (/home/jovyan/.cache/huggingface/datasets/anton-l___superb_demo/er/1.9.0/77d23894ff429329a7fe80f9007cabb0deec321316f8dda1a1e9d10ffa089d08)
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/anton-l___superb_demo/er/1.9.0/77d23894ff429329a7fe80f9007cabb0deec321316f8dda1a1e9d10ffa089d08/cache-374cee6acfc77828.arrow


In [2]:

model = HubertForSequenceClassification.from_pretrained("superb/hubert-large-superb-er")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/hubert-large-superb-er")


In [6]:

# compute attention masks and normalize the waveform if needed
inputs = feature_extractor(dataset[:2]["speech"], sampling_rate=16000, padding=True, return_tensors="pt")


In [45]:
print(type(dataset[:2]['speech'][0]))
print(dataset[0]['audio']['array'][:4])
print(len(dataset[:2]['audio']))
print(dataset[:2]['speech'][0][:4])
print(dataset[:2]['audio'][0])

<class 'list'>
[ 0.01025391  0.00134277 -0.0039978  -0.00628662]
2
[0.01025390625, 0.0013427734375, -0.003997802734375, -0.00628662109375]
{'path': '/home/jovyan/.cache/huggingface/datasets/downloads/extracted/78e6da90361e53b909a57c3b6dc0334cd09f6bc67691afb514a961f5df6e9bdf/Session1/sentences/wav/Ses01F_impro03/Ses01F_impro03_F013.wav', 'array': array([ 0.01025391,  0.00134277, -0.0039978 , ..., -0.00027466,
       -0.00231934, -0.00259399], dtype=float32), 'sampling_rate': 16000}


In [46]:
alt_input = feature_extractor([audio['array'] for audio in dataset[:2]['audio']], sampling_rate=16000, padding=True, return_tensors="pt")

In [47]:

# logits = model(**inputs).logits
logits = model(**alt_input).logits


In [48]:

predicted_ids = torch.argmax(logits, dim=-1)
labels = [model.config.id2label[_id] for _id in predicted_ids.tolist()]